In [1]:
from __future__ import annotations

import cProfile
import pstats
from asyncio import sleep

In [2]:
from py_research.db import (
    DataSource,
    TableMap,
    SubTableMap,
    SubMap,
    SelIndex,
    DataBase
)

In [3]:
from schema import (
    Search,
    Project,
    Task,
    User,
    Membership,
    Organization,
    SearchResult,
    TestSchema,
)

In [4]:
from typing import Any


async def _dummy_loader(obj: dict[str, Any]) -> dict:
    await sleep(0.2)
    return {
        'name': obj["name"],
        'role': obj["role"],
        'age': obj["age"] + 1
    }

In [5]:
data_source = DataSource(
    target=Search,
    push={
        "resultCount": Search.result_count,
        "search": Search.term,
        "results": SubTableMap(
            target=Search.results,
            rel_map=TableMap(push={SearchResult.score}),
            push={
                "project_name": Project.name,
                "project_start": Project.start,
                "project_end": Project.end,
                "project_status": Project.status,
                "tasks": SubTableMap(
                    target=Project.tasks,
                    push={
                        "task_name": Task.name,
                        "task_assignees": SubTableMap(
                            target=Task.assignees,
                            push=User.name,
                            match_by=[User.name],
                        ),
                        "task_status": Task.status,
                    },
                ),
                "members": SubTableMap(
                    target=Project.members,
                    loader=_dummy_loader,
                    push={User.name, User.age},
                    match_by=[User.name],
                    rel_map=TableMap(
                        push={
                            Membership.role,
                        },
                    ),
                ),
            },
            pull={
                Project.org: SubMap(
                    push={
                        "organization_name": Organization.name,
                        "organization_address": Organization.address,
                        "organization_city": Organization.city,
                        "organization_countries": Organization.ccs,
                    },
                    match_by=[Organization.name, Organization.city],
                ),
                Project.number: SelIndex()
            },
        ),
    },
)

In [6]:
from datetime import date


data = {
    "resultCount": 3,
    "search": "test",
    "results": [
        {
            "score": 0.9,
            "project_name": "baking cake",
            "project_start": date(2020, 1, 1),
            "project_end": date(2020, 1, 4),
            "project_status": "done",
            "organization_name": "Bakery",
            "organization_address": "Main Street 1",
            "organization_city": "Bakerville",
            "organization_countries": ["USA", "Mexico", "Canada"],
            "tasks": [
                {
                    "task_name": "task1",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task2",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task3",
                    "task_assignees": ["Jane"],
                    "task_status": "done",
                },
            ],
            "members": [
                {"name": "John", "role": "baker", "age": 30},
                {"name": "John", "role": "manager", "age": 40},
            ],
        },
        {
            "score": 0.8,
            "project_name": "cleaning shoes",
            "project_start": date(2020, 1, 2),
            "project_end": date(2020, 1, 5),
            "project_status": "done",
            "organization_name": "Shoe Shop",
            "organization_address": "Main Street 2",
            "organization_city": "Shoetown",
            "organization_countries": ["USA", "Mexico"],
            "tasks": [
                {
                    "task_name": "task4",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task5",
                    "task_assignees": ["Jane"],
                    "task_status": "todo",
                },
            ],
            "members": [
                {"name": "John", "role": "cleaner", "age": 25},
                {"name": "Jane", "role": "manager", "age": 35},
            ],
        },
        {
            "score": 0.7,
            "project_name": "fixing cars",
            "project_start": date(2020, 1, 3),
            "project_end": date(2020, 1, 6),
            "project_status": "started",
            "organization_name": "Car Shop",
            "organization_address": "Main Street 3",
            "organization_city": "Cartown",
            "organization_countries": ["USA", "Canada"],
            "tasks": [
                {
                    "task_name": "task6",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                }
            ],
            "members": [
                {"name": "John", "role": "mechanic", "age": 45},
                {"name": "Jane", "role": "manager", "age": 55},
                {"name": "Jack", "role": "manager", "age": 65},
            ],
        },
    ],
}

In [7]:
from pathlib import Path
import warnings

with warnings.catch_warnings():
  warnings.simplefilter("error")
  with cProfile.Profile() as pr:
    db = DataBase(
      schema=TestSchema,
      url=Path("./test.xlsx")
    )
    rec = await data_source.load([data], db=db)

    stats = pstats.Stats(pr)

In [9]:
db.describe()


{
    'schema': {
        'package': 'py-research',
        'module': 'schema',
        'object': 'TestSchema',
        'package_version': '^5.1.1.dev152+g2c0bcb5.d20250123',
        'repo': 'https://github.com/cloudlane-one/py-research.git',
        'repo_revision': '2c0bcb5afbc13db112553eaeb6ece6863d7a5a2f',
        'module_path': WindowsPath('exp/schema.py'),
        'module_dirty': False,
        'docs_url': 'https://cloudlane-one.github.io/py-research/'
    },
    'contents': {
        'records': {
            'schema.Organization': 3,
            'schema.Task': 6,
            'schema.Search': 1,
            'schema.Project': 3,
            'schema.User': 3
        },
        'arrays': {'schema.Organization_ccs': 7},
        'relations': {'schema.Membership': 7, 'schema.SearchResult': 3, 'schema.Assignment': 6}
    },
    'backend': 'test.xlsx'
}

In [10]:
nodes, edges = db.to_graph([Search, Project, Organization])

In [11]:
nodes

,node_id,schema_Search_term,schema_Search_result_count,table,schema_Project_number,schema_Project_name,schema_Project_start,schema_Project_end,schema_Project_status,schema_Project_org__id,schema_Organization__id,schema_Organization_name,schema_Organization_address,schema_Organization_city
0,0,test,3.0,schema_Search,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,schema_Project,0.0,baking cake,2020-01-01,2020-01-04,done,6cc1eaeb-8c93-4040-9d6c-156f32eeac98,NaN,NaN,NaN,NaN
2,2,NaN,NaN,schema_Project,1.0,cleaning shoes,2020-01-02,2020-01-05,done,fd6c93de-5cd0-4bbf-9c72-01bd85c76ee3,NaN,NaN,NaN,NaN
3,3,NaN,NaN,schema_Project,2.0,fixing cars,2020-01-03,2020-01-06,started,01d9df67-4b7d-4ff5-a50b-5127fe47a056,NaN,NaN,NaN,NaN
4,4,NaN,NaN,schema_Organization,NaN,NaN,NaN,NaN,NaN,NaN,fd6c93de-5cd0-4bbf-9c72-01bd85c76ee3,Bakery,Main Street 1,Bakerville
5,5,NaN,NaN,schema_Organization,NaN,NaN,NaN,NaN,NaN,NaN,01d9df67-4b7d-4ff5-a50b-5127fe47a056,Shoe Shop,Main Street 2,Shoetown
6,6,NaN,NaN,schema_Organization,NaN,NaN,NaN,NaN,NaN,NaN,6cc1eaeb-8c93-4040-9d6c-156f32eeac98,Car Shop,Main Street 3,Cartown


In [12]:
edges

,source,target,ltr,rtl,schema_SearchResult__to_number,schema_SearchResult__id,schema_SearchResult__from_term,schema_SearchResult_score
0,1,6,org__id,None,NaN,NaN,NaN,NaN
1,2,4,org__id,None,NaN,NaN,NaN,NaN
2,3,5,org__id,None,NaN,NaN,NaN,NaN
3,0,1,_to_number,_from_term,0.0,4376575091,test,0.9
4,0,3,_to_number,_from_term,2.0,3380659540,test,0.8
5,0,2,_to_number,_from_term,1.0,3910210571,test,0.7


In [13]:
db[User].values()


[
    User({'_id': 'a8e9b430-4e2e-4178-8e69-8116f006a46c', 'name': 'John', 'age': 31}),
    User({'_id': 'e23b57cf-025e-4cdd-b418-5ede74565dce', 'name': 'Jane', 'age': 56}),
    User({'_id': 'd58faa3c-2d3d-4639-a62d-0b3d184bc2db', 'name': 'Jack', 'age': 66})
]

In [14]:
db[Project].df()

number,name,start,end,status,org__id
i64,str,date,date,str,str
0,"""baking cake""",2020-01-01,2020-01-04,"""done""","""6cc1eaeb-8c93-4040-9d6c-156f32…"
1,"""cleaning shoes""",2020-01-02,2020-01-05,"""done""","""fd6c93de-5cd0-4bbf-9c72-01bd85…"
2,"""fixing cars""",2020-01-03,2020-01-06,"""started""","""01d9df67-4b7d-4ff5-a50b-5127fe…"


In [15]:
db[Search.term].values()

['test']

In [16]:
list(db[Search.result_count].items())

[('test', 3)]

In [17]:
list(db[Search.results.x.org].items())


[
    (
        ('test', 0, '6cc1eaeb-8c93-4040-9d6c-156f32eeac98'),
        Organization({'_id': '6cc1eaeb-8c93-4040-9d6c-156f32eeac98', 'name': 'Car Shop', 'address': 'Main Street 3', 'city': 'Cartown'})
    ),
    (
        ('test', 2, '01d9df67-4b7d-4ff5-a50b-5127fe47a056'),
        Organization({'_id': '01d9df67-4b7d-4ff5-a50b-5127fe47a056', 'name': 'Shoe Shop', 'address': 'Main Street 2', 'city': 'Shoetown'})
    ),
    (
        ('test', 1, 'fd6c93de-5cd0-4bbf-9c72-01bd85c76ee3'),
        Organization({'_id': 'fd6c93de-5cd0-4bbf-9c72-01bd85c76ee3', 'name': 'Bakery', 'address': 'Main Street 1', 'city': 'Bakerville'})
    )
]

In [18]:
list(db[Search.results].items())[0][1].start.year

2020

In [19]:
list(db[Search.results]._abs_cols.keys())

['number', 'name', 'start', 'end', 'status', 'org__id']

In [20]:
db[Search.results].df()

schema_Search_term,number,name,start,end,status,org__id
str,i64,str,date,date,str,str
"""test""",0,"""baking cake""",2020-01-01,2020-01-04,"""done""","""6cc1eaeb-8c93-4040-9d6c-156f32…"
"""test""",2,"""fixing cars""",2020-01-03,2020-01-06,"""started""","""01d9df67-4b7d-4ff5-a50b-5127fe…"
"""test""",1,"""cleaning shoes""",2020-01-02,2020-01-05,"""done""","""fd6c93de-5cd0-4bbf-9c72-01bd85…"


In [21]:
db[Search.results.x.org].df()

schema_Search_term,schema_Search_results_number,_id,name,address,city
str,i64,str,str,str,str
"""test""",0,"""6cc1eaeb-8c93-4040-9d6c-156f32…","""Car Shop""","""Main Street 3""","""Cartown"""
"""test""",2,"""01d9df67-4b7d-4ff5-a50b-5127fe…","""Shoe Shop""","""Main Street 2""","""Shoetown"""
"""test""",1,"""fd6c93de-5cd0-4bbf-9c72-01bd85…","""Bakery""","""Main Street 1""","""Bakerville"""


In [22]:
db[Organization].df()

_id,name,address,city
str,str,str,str
"""fd6c93de-5cd0-4bbf-9c72-01bd85…","""Bakery""","""Main Street 1""","""Bakerville"""
"""01d9df67-4b7d-4ff5-a50b-5127fe…","""Shoe Shop""","""Main Street 2""","""Shoetown"""
"""6cc1eaeb-8c93-4040-9d6c-156f32…","""Car Shop""","""Main Street 3""","""Cartown"""


In [23]:
db[SearchResult._table[SearchResult.score.isin(slice(0.81, 0.9))]].df()

_id,score,_from_term,_to_number
str,f64,str,i64
"""4376575091""",0.9,"""test""",0


In [24]:
import pandas as pd


db[Organization.ccs].df(pd.DataFrame)

value
schema_Organization__id              schema_Organization_ccs_idx        
6cc1eaeb-8c93-4040-9d6c-156f32eeac98 0                               USA
                                     1                            Mexico
                                     2                            Canada
fd6c93de-5cd0-4bbf-9c72-01bd85c76ee3 0                               USA
                                     1                            Mexico
01d9df67-4b7d-4ff5-a50b-5127fe47a056 0                               USA
                                     1                            Canada

In [25]:
stats.sort_stats("cumulative")
stats.print_stats("py_research")

         1220731 function calls (1181003 primitive calls) in 3.941 seconds

   Ordered by: cumulative time
   List reduced from 2795 to 163 due to restriction <'py_research'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       55    0.000    0.000    2.222    0.040 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:697(_load_rec_from_item)
       55    0.002    0.000    2.222    0.040 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:572(_load_record)
       19    0.000    0.000    2.178    0.115 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:1504(__ior__)
       19    0.001    0.000    2.178    0.115 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:2948(_mutate)
       18    0.001    0.000    2.018    0.112 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:3085(_mutate_from_records)
       16    0.003    0.000    1.987    0.124 C:\Users\lworm\Repos\py-research\src\py_research\